# Imports

In [278]:
# Usual stuff
import numpy as np
import matplotlib.pyplot as plt

# Typing
from typing import Callable

## Units

We are going to use units where:
- $t = 1$
- $k_B = 1$

# Numerical Methods

In [279]:
def nascent_delta(x: float | np.ndarray, gam: float) -> float | np.ndarray:
    return np.exp(-x * x / (gam * gam)) / (gam * np.sqrt(np.pi))

In [280]:
def bissect(func: Callable, a: float, b: float, eps: float = 1e-6, maxI: int = 200) -> float:
    """
        Solves the equation `func(x) = 0` using the bissection method in [a, b].

        The function stops when the error is smaller then `eps` or the number of iterations exceeds `maxI`.

        ### Return
        x: The value of x that solves the equation.
    """

    # Evaluate on the edges
    fa, fb = func(a), func(b)

    # Check if there is a zero on the edges
    if abs(fa) <= 1e-16:
        return a
    elif abs(fb) <= 1e-16:
        return b

    # Main Loop
    i = 0
    error = (b-a)/2
    while abs(error) > eps:

        # Find the midpoint and evaluate the function there
        c = (a+b)/2
        fc = func(c)

        # Check if the zero is exactly on the midpoint
        if abs(fc) <= 1e-16:
            return c

        # Sign change happens in [a, c]
        elif fa * fc < 0:
            b = c
            fb = fc
        
        # Sign change happens in [c, b]
        elif fc * fb < 0:
            a = c
            fa = fc
        
        # No sign change
        else:
            raise ValueError(f"O sinal de f é o mesmo em a = {a} e em b = {b} (f(a) = {fa}, f(b) = {fb}) - iteração {i}).")

        # Compute the error
        error = (b - a)/2

        # Maximum number of iterations
        i += 1
        if i > maxI:
            raise ValueError(f"Não foi possível encontrar a raíz da função com a precisão desejada em menos de maxI = {maxI} iterações!")
        
    return (a+b)/2

# Physics

In [281]:
def energy(kk: np.ndarray):
    """
        Compute the dispersion relation for a given vector `k`.
    """
    return -2 * np.sum(np.cos(kk), axis = 1)

## Fermi Energy $E_F$

We compute the following Formula using Monte Carlo integration:
$$
\begin{align*}
n &= \int_\text{FBZ} \frac{d^d\textbf k}{(2\pi)^d}\  \theta(U_c - (\varepsilon_\textbf k - \varepsilon_\text{min}))
\end{align*}
$$

In [282]:
def fermi(d: int, n: float, mult: int = 2, biss_eps: float = 5e-3, mc_eps: float = 1e-3):
    """
        Compute the Fermi energy or critical interaction `Uc` for `d` dimensions with filling `n`.

        If `mult` == 2 then we obtain the usual Fermi energy.
        If `mult` == 1 then we compute the critical interaction strength.

        Uses the bissection method with error `biss_eps` and Monte Carlo integration with error `mc_eps`.
    """


    def n_test(Uc: float):
        """
            Apply the formula above and determine the values of n predicted by Uc.
        """

        # Initial number of sampled points
        Ns = 100

        # Compute the integral with Ns samples
        k_Ns = 2 * np.pi * (np.random.random((Ns, d)) - 0.5)
        E_Ns = energy(k_Ns)
        I_Ns = np.sum(mult * np.heaviside(Uc - (E_Ns + 2 * d), 0.5)) / Ns

        # Main loop
        eps = mc_eps + 1
        while eps > mc_eps:
            
            # Increase number of samples
            Ns *= 4
            
            # Compute the integral with 4*Ns samples
            k_Ns = 2 * np.pi * (np.random.random((Ns, d)) - 0.5)
            E_Ns = energy(k_Ns)
            I_4Ns = np.sum(mult * np.heaviside(Uc - (E_Ns + 2 * d), 0.5)) / Ns

            # Estimate the error 
            eps = np.abs(I_4Ns - I_Ns)

            # Update for next step
            I_Ns = I_4Ns
        
        return I_4Ns
    
    # Find Uc via bissecion
    Uc = bissect(lambda x: n_test(x) - n, 0, 4 * d, eps = biss_eps)

    return Uc


# print(f"Critical U for n = 0.5 and d = 1: {fermi(1, 0.5, 1)}")
# print(f"Critical U for n = 1.0 and d = 2: {fermi(2, 1.0, 1)}")
# print("")
# print(f"Fermi Energy for n = 0.5 and d = 1: {fermi(1, 0.5, 2, mc_eps=0.25e-2)}")
# print(f"Fermi Energy for n = 1.0 and d = 2: {fermi(2, 1.0, 2, mc_eps=0.25e-2)}")

## Energy $\varepsilon_0$

In [283]:
def eps0(d: int, n: float, U: float, biss_eps: float = 10e-3, mc_eps: float = 4e-3):
    """
        Compute `eps0` for `d` dimensions with filling `n`.

        Uses the bissection method with error `biss_eps` and Monte Carlo integration with error `mc_eps`.
    """


    def n_test(eps0: float):
        """
            Apply the formula above and determine the values of n predicted by Uc.
        """

        # Initial number of sampled points
        Ns = 100

        # Compute the integral with Ns samples
        k_Ns = 2 * np.pi * (np.random.random((Ns, d)) - 0.5)
        E_Ns = energy(k_Ns)
        I_Ns = np.sum(np.heaviside(eps0 + U - (E_Ns + 2 * d), 0.5) + np.heaviside(eps0 - (E_Ns + 2 * d), 0.5)) / Ns

        # Main loop
        eps = mc_eps + 1
        while eps > mc_eps:
            
            # Increase number of samples
            Ns *= 4
            
            # Compute the integral with 4*Ns samples
            k_Ns = 2 * np.pi * (np.random.random((Ns, d)) - 0.5)
            E_Ns = energy(k_Ns)
            I_4Ns = np.sum(np.heaviside(eps0 + U - (E_Ns + 2 * d), 0.5) + np.heaviside(eps0 - (E_Ns + 2 * d), 0.5)) / Ns

            # Estimate the error 
            eps = np.abs(I_4Ns - I_Ns)

            # Update for next step
            I_Ns = I_4Ns
        
        return I_4Ns
    
    # Find eps0 via bissecion
    eps0 = bissect(lambda x: n_test(x) - n, 0, 4 * d, eps = biss_eps)

    return eps0


# print(f"Eps0 for U = 1, n = 0.5 and d = 1: {eps0(1, 0.5, 1)}")
# print(f"Eps0 for U = 2, n = 1.0 and d = 1: {eps0(1, 1.0, 2)}")

# Main Code

We will compute the critical temperature, by determining at which temperature $\chi_0(0) = 1/g$:
$$
\begin{align*}

\chi_0(0) &= \int d\omega\ N'(\omega) \cdot \frac{\tanh \frac{\beta \omega}{2}}{2\omega}

\end{align*}
$$

Where:
- $N'(\omega)$ is an effective density of states:
    $$
    \begin{align*}
    
    N'(\omega) &= \frac1{L_d}\left(\sum_{\textbf k \in \Omega_0} \delta(\omega - \xi_\textbf k^l) + \sum_{\textbf k \in \Omega_2} \delta(\omega - \xi_\textbf k^u) + \frac14\sum_{\textbf k \in \Omega_1} \delta(\omega - \xi_\textbf k^l)  +\delta(\omega - \xi_\textbf k^u) \right)
    
    \end{align*}
    $$
    
- $\xi_\textbf k$ is given by:
    $$
    \begin{align*}
    
    \xi_\textbf k^u &= \xi_\textbf k + U
    \\\\
    
    \xi_\textbf k^l &= \xi_\textbf k
    
    \end{align*}
    $$

## Zero Temperature calculation of $\lang n_{\textbf k \sigma}\rang$

At $T=0$ we have:
- For $U < 0$ we have:
    $$
        \begin{align*}
            \langle n_{k\uparrow} \rangle = \langle n_{k\downarrow} \rangle =
            \begin{cases}
            1 & \varepsilon_k \le E_F \\[6pt]
            0 & \text{otherwise}
            \end{cases}
        \end{align*}
    $$

- For $0 < U < U_c$ we have:
    $$
        \begin{align*}
            \langle n_{k\uparrow} \rangle = \langle n_{k\downarrow} \rangle =
            \begin{cases}
            1 & \varepsilon_k \le \varepsilon_0 \\[6pt]
            \frac{1}{2} & \varepsilon_0 < \varepsilon_k \le \varepsilon_0 + U \\[6pt]
            0 & \varepsilon_0 + U < \varepsilon_k
            \end{cases}
        \end{align*}
    $$

- For $U > U_c$ we have:
    $$
        \begin{align*}

            \langle n_{k\uparrow} \rangle = \langle n_{k\downarrow} \rangle =
            \begin{cases}

            \frac{1}{2} & \varepsilon_\textbf k \le U_c \\[6pt]
            0 & \text{otherwise}
            \end{cases}

        \end{align*}
    $$

In [284]:
def bigomegaT0(d: int, n: float, U: float, mesh_ek: np.ndarray, mesh_kk: np.ndarray):
    """
        Compute the regions of the FBZ which have occupation 0, 1 or 2.
    """

    # Bandwith and minimum energy
    W = 4 * d
    emin = -W/2

    # Pre-compute model properties
    Ef = fermi(d, n, 2)
    Uc = fermi(d, n, 1)
    e0 = eps0(d, n, U)

    # Difference from energy minimum
    mesh_deltaek = mesh_ek - emin

    # Transpose to save in the omega lists
    mesh_kk = mesh_kk.T

    # Omega regions
    omega_0 = np.zeros_like(mesh_ek)
    omega_1 = np.zeros_like(mesh_ek)
    omega_2 = np.zeros_like(mesh_ek)

    
    # Attractive Case
    if U <= 0:

        # If ek <= Ef then Omega = 2 else Omega = 0
        omega_2 = (mesh_deltaek <= Ef)
        omega_0 = ~omega_2
    

    # Weakly Repeling case
    elif U < Uc:
        
        # If ek <= e0 then Omega = 2 else if e0 < ek < e0 + U then OMega = 1 else Omega = 0
        omega_2 = (mesh_deltaek <= e0)
        omega_1 = ((e0 < mesh_deltaek) & (mesh_deltaek < (e0 + U)))
        omega_0 = ~(omega_2 | omega_1)
        
    
    # Strongly Repeling case
    else:
        
        # If ek <= Uc then Omega = 1 else Omega = 0
        omega_1 = (mesh_deltaek <= Uc)
        omega_0 = ~(omega_1)

    return omega_0, omega_1, omega_2


# omega_0, omega_1, omega_2 = bigomega(1, 1, 12, 2)

In [285]:
def TcT0(d: int, n: float, V: int, U: float, gam: float, Wnum: int, g: float, biss_eps: float = 1e-6):
    """
        Computes Tc using the T = 0 regions Omega0, Omega1 and Omega2
    """

    # STAGE 0: Preparation

    # Bandwith and minimum energy
    W = 4 * d

    # Determine values of k in our lattice
    L = int(np.ceil(V**(1/d)))
    kk = np.linspace(-np.pi, np.pi, L + 1, retstep=False)[:-1]

    # Make a mesh of k values
    grid_kk = np.meshgrid(*[kk]*d, indexing='ij')

    # Convert to a single array with shape (d, len(kk) ** d)
    mesh_kk = np.stack(grid_kk).reshape(d, len(kk) ** d).T

    # Get the energy for each k
    mesh_ek = energy(mesh_kk)

    # Update V according to rounding
    V = len(mesh_ek)

    # Grid in omega
    Wmin = U - W/2 - 3
    Wmax = U + W/2 + 3

    mesh_ww = np.linspace(Wmin, Wmax, Wnum)

    # Make a mesh with points (ek, omega)
    mesh_cek, mesh_cww = np.meshgrid(mesh_ek, mesh_ww)

    # Brillouin zone areas
    omega_0, omega_1, omega_2 = bigomegaT0(d, n, U, mesh_ek, mesh_kk)


    # STAGE 1: Compute A(omega)

    # K-dependent spectral function
    Akw = (1.0 * (omega_0 > 0) + 0.5 * (omega_1 > 0)) * nascent_delta(mesh_cww - mesh_cek, gam) + (1.0 * (omega_2 > 0) + 0.5 * (omega_1 > 0)) * nascent_delta(mesh_cww - mesh_cek - U, gam)
    
    # Sum along K-axis
    Aw = np.sum(Akw, axis=1)


    # STAGE 2: Compute mu

    # Integrate DOS 
    h = mesh_ww[1] - mesh_ww[0]
    trap = (Aw[:-1] + Aw[1:]) / 2
    idos = h * np.concatenate((np.array([0]), trap.cumsum()))

    # Normalize to 2
    ndos = (2 / idos[-1])
    idos *= ndos

    # Highest filled state and lowest empty state
    filled = mesh_ww[idos <= n]
    empty = mesh_ww[idos >= n]

    # Determine chemical potential at T = 0
    # It is midway between the last filled level and last unfilled level
    mu = 0.5 * (filled[-1] + empty[0])

    # Update energy dispersion
    mesh_xk = mesh_ek - mu

    print(f"mu = {mu}")
    print(f"len0 = {np.sum(omega_0)} | len1 = {np.sum(omega_1)} | len2 = {np.sum(omega_2)}")


    # Do the function tanh(x)/x using the correct limit as x -> 0
    def tanhoverx(beta: float, xx: np.ndarray):
        
        # Mask for small values of x
        small = np.isclose(xx, 0.0)

        # Output (normal for ~small and analytic limit for small)
        out = np.empty_like(xx, dtype=float)
        out[~small] = np.tanh(beta * xx[~small] / 2.0) / (2.0 * xx[~small])
        out[small] = beta / 4.0

        return out
    

    def chi0T0(T: float):
        
        # Thermodynamic temperature
        beta = 1/T

        # STAGE 3: Compute chi0(0)
        chi0l = np.sum((1.0 * (omega_0 > 0) + 0.25 * (omega_1 > 0)) * tanhoverx(beta, mesh_xk))
        chi0u = np.sum((1.0 * (omega_2 > 0) + 0.25 * (omega_1 > 0)) * tanhoverx(beta, mesh_xk + U))

        return (chi0l + chi0u) / V


    # Compute Tc
    Tc = bissect(lambda T: chi0T0(T) - 1/g, 1e-9, 1e-2, biss_eps)

    return Tc



# PARAMETERS

# Model
d = 1
n = 1
V = 6000

W = 4 * d
U = 0.6 * W
g = 0.1 * W


# Calculations 
gam = 0.03
Wnum = 2000

# print(TcT0(d, n, V, U, gam, Wnum, g))

# out:
# mu = 1.1893946973486742
# len0 = 1803 | len1 = 2458 | len2 = 1739
# 0.0001385507908325195

## Finite Temperature calculation of $\lang n_{\textbf k \sigma}\rang$

In [286]:
def bigomegaT0(U: float, mesh_xk: np.ndarray):
    """
        Compute the regions of the FBZ which have occupation 0, 1 or 2 at T = 0.
    """

    # Omega regions
    mesh_nk = np.zeros_like(mesh_xk)
    omega_0 = np.zeros_like(mesh_xk)
    omega_1 = np.zeros_like(mesh_xk)
    omega_2 = np.zeros_like(mesh_xk)

    # We have double occupation if UHB is below chemical potential
    omega_2 = (mesh_xk + U) < 0

    # We have empty states if LHB is above the chemical potential
    omega_0 = mesh_xk > 0

    # Otherwise we have single-filled states
    omega_1 = ~(omega_0 | omega_2)

    # Build mesh_nk
    mesh_nk[omega_0] = 0
    mesh_nk[omega_1] = 1/2
    mesh_nk[omega_2] = 1


    return mesh_nk, omega_0, omega_1, omega_2


def occupations(U: float, beta: float, mesh_xk: np.ndarray, cutoff: float = 30):
    """
        Compute the average occupation of each state.
    """

    # Omega regions
    mesh_nk = np.zeros_like(mesh_xk)

    # Arguments of exponentials
    arg_zk = -beta * mesh_xk
    arg_zk1U = -beta * (mesh_xk + U)
    arg_zk2U = -beta * (2 * mesh_xk + U)

    # Determine when the approximate formula is required
    approx = arg_zk > cutoff

    # Clip to prevent overflow errors
    arg_zk = np.clip(arg_zk, -700, 700)
    arg_zk1U = np.clip(arg_zk1U, -700, 700)
    arg_zk2U = np.clip(arg_zk2U, -700, 700)

    # Use the approximate formula
    mesh_nk[approx] = 1 - 1 / (2 + np.exp(arg_zk1U[approx]))

    # Compute log(1 - nk)
    mesh_zk = np.exp(arg_zk)
    mesh_nk[~approx] = np.log1p(mesh_zk[~approx]) - np.log1p(2 * mesh_zk[~approx] + np.exp(arg_zk2U[~approx]))

    # Compute nk
    mesh_nk[~approx] = 1 - np.exp(mesh_nk[~approx])

    return mesh_nk

In [287]:
# Do the function tanh(x)/x using the correct limit as x -> 0
def tanhoverx(beta: float, xx: np.ndarray):
    
    # Mask for small values of x
    small = np.abs(xx) < 1e-12

    # Output (normal for ~small and analytic limit for small)
    out = np.empty_like(xx, dtype=float)
    out[~small] = np.tanh(beta * xx[~small] / 2.0) / (2.0 * xx[~small])
    out[small] = beta / 4.0

    return out

In [ ]:
def TcTf(d: int, n: float, V: int, U: float, g: float, betai: float = 10000.0, mu_eps: float = 1e-3, Tc_eps: float = 1e-2):
    """
        Computes Tc using finite temperature values of nk.

        The error parameters are:
        - `mu_eps` is the linear error on the chemical potential
        - `Tc_eps` is the multiplicative error on the critical temperature
    """

    # STAGE 0: Preparation

    # Bandwith and minimum energy
    W = 4 * d

    # Determine values of k in our lattice
    L = int(np.ceil(V**(1/d)))
    kk = np.linspace(-np.pi, np.pi, L + 1, retstep=False)[:-1]

    # Make a mesh of k values
    grid_kk = np.meshgrid(*[kk]*d, indexing='ij')

    # Convert to a single array with shape (d, len(kk) ** d)
    mesh_kk = np.stack(grid_kk).reshape(d, len(kk) ** d).T

    # Get the energy for each k
    mesh_ek = energy(mesh_kk)

    # Update V according to rounding
    V = len(mesh_ek)


    # TODO: Change to outer bissection for Tc and inner bissection for mu

    # LOOP: Solve self-consistently for Tc 
    beta = betai
    Tc = 1/betai

    Tc_error = Tc_eps + 1
    while Tc_error > Tc_eps:

        # STEP 1: Solve self-consistently for mu at fixed beta

        # Minimum and maximum mu are below the bands by T (so that they correspond to completely empty and completely full)
        mu_min = -U - W - 1/beta
        mu_max = U + W + 1/beta

        # Compute total occupation n(mu)
        def n_test(mu: float):

            # Occupation numbers
            mesh_nk = occupations(U, beta, mesh_ek - mu)

            return 2 * np.sum(mesh_nk) / V
    
        # Determine mu via bissection
        mu = bissect(lambda x: n_test(x) - n, mu_min, mu_max, mu_eps)


        # STEP 2: Determine Tc for this beta

        # Occupation numbers
        mesh_nk = occupations(U, beta, mesh_ek - mu)

        # Plot (debugging)
        # print(f"mu = {mu}")
        # plt.plot(mesh_ek, mesh_nk)
        # plt.title("nk")
        # plt.show()


        # Compute l and u terms
        mesh_nu = mesh_nk
        mesh_nl = 1 - mesh_nk

        mesh_xl = mesh_ek - mu
        mesh_xu = mesh_ek - mu + U

        def chi0(T: float):
            
            # Thermodynamic temperature
            beta = 1/T

            # Compute direct terms
            chi_ll = mesh_nl * mesh_nl * tanhoverx(beta, mesh_xl)
            chi_uu = mesh_nu * mesh_nu * tanhoverx(beta, mesh_xu)

            # Compute cross terms
            # arg_ul = np.log1p(-np.exp(np.clip(-beta * (2 * mesh_xl + U), -700, 700))) - np.log1p(np.exp(np.clip(-beta * mesh_xl, -700, 700))) - np.log1p(np.exp(np.clip(-beta * mesh_xu, -700, 700)))
            # arg_ul = np.clip(arg_ul, -700, 700)

            # num_ul = np.exp(arg_ul)
            # chi_ul = 2 * mesh_nl * mesh_nu * num_ul / (2 * mesh_xl + U)

            # print(f"chi_ll = {np.sum(chi_ll)}, chi_uu = {np.sum(chi_uu)}, chi_ul = {np.sum(chi_ul)}")

            return (np.sum(chi_ll) + np.sum(chi_uu)) / V


        # Compute Tc (error in bissection has to be one order of magnitude smaller then error in Tc)
        Tc_new = bissect(lambda T: chi0(T) - 1/g, 1e-9, 1e-1, Tc * Tc_eps * 0.2)


        # Update
        Tc_error = np.abs(Tc_new - Tc)

        Tc = Tc_new
        beta = 1/Tc


    return Tc



# PARAMETERS

# Model
d = 1
n = 1
V = 6000

W = 4 * d
U = 0.6 * W
g = 0.1 * W

Tc = TcTf(d, n, V, U, g) / W

print(f"Tc = {Tc}")


# Plot
xx = [0, 0.02, 0.04, 0.06, 0.1]
tt = np.zeros_like(xx)
for index, x in enumerate(xx):
    print(f"index = {index} and x = {x}")
    
    n = 1 - x
    tt[index] = TcTf(d, n, V, U, g) / W



plt.plot(xx, tt)
plt.title("Critical Temperature")
plt.xlabel("x (Hole Doping)")
plt.ylabel("Tc / W")
plt.yscale("log")

Tc = 3.318811587905884e-05
index = 0 and x = 0
index = 1 and x = 0.02


ValueError: O sinal de f é o mesmo em a = 1e-09 e em b = 0.1 (f(a) = -0.1826279999020124, f(b) = -1.7002034403551747) - iteração 0).